In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [2]:
# Database Setup
engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
df_gunviolence = pd.read_sql('select * from gunviolence_db', engine)
print(len(df_gunviolence))

425


In [3]:
#@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

In [10]:
###### QUESTION: why need to change the variable list_states -> list_states2?

#@app.route("/states")
def list_states():
    """Return a list of states."""

    # Use Pandas to perform the sql query
    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gunviolence = pd.read_sql('select State from gunviolence_db', engine)
    
    stmt = list(df_gunviolence.sort_values(by=['State']).unique())

    # Return a list of the column names (state names)
    #return jsonify(stmt)
    return stmt

states = list_states()
print(states)

AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
# METADATA TABLE: Create a summary of gun violence by state. 
# Changed df_gun_violence2 -> df_gun_violence_meta

@app.route("/metadata/<state>")
def gunviolence_metadata(state):
    """Return the gun violence data summary (metadata) for a given state."""

    engine = create_engine("sqlite:///db/gunviolence_db.sqlite")
    df_gunviolence_meta = pd.read_sql(f"select * from gunviolence_db where State = '{state}' ", engine)
    results = [
        list(df_gunviolence_meta['State']),
        list(df_gunviolence_meta['year']),
        list(df_gunviolence_meta['Incident ID']),
        list(df_gunviolence_meta['Killed']),
        list(df_gunviolence_meta['Injured'])
    ]
    print('results:', results)
            
    # Create a dictionary entry for each row of gun violence summary data (metadata)
    gun_violence_metadata = '{
        df_gunviolence_metadata.to_json(orient='index')
        "year":{(df_gunviolence_meta['year'])},
        "Incident_ID":{(df_gunviolence_meta['Incident ID']).count}, 
        "Number_Killed":{(df_gunviolence_meta['Killed']).sum},
        "Number_Injured":{(df_gunviolence_meta['Injured']).sum}
    }'
    
    print(sample_metadata)
    return jsonify(sample_metadata)


#>>> df.to_json(orient='index')
#'{"row 1":{"col 1":"a","col 2":"b"},"row 2":{"col 1":"c","col 2":"d"}}'

In [ ]:
# GOAL: Create a dictionary entry for each row of gun violence summary data (metadata)
# { 'year' : 2018
#   'incident_count': "IncidentID".count,
#   'number_killed': "Killed".sum,
#   'number_injured': "Injured".sum
# }